In [19]:
import pandas as pd
import tensorflow as tf
from glob import glob
import numpy as np
from easydict import EasyDict as edict
import os
import time
import sys

sys.path.insert(0,os.path.expanduser('~/git/torchseg'))
print(sys.path)

from utils.config import load_config
from utils.summary_to_csv import config_to_log,load_log,edict_to_pandas

def summary(rootpath,tags,filter_str=None,):
    config_files=glob(os.path.join(rootpath,'**','config.txt'),recursive=True)
    if filter_str is not None:
        config_files=[f for f in config_files if f.find(filter_str)>=0]
    
    tasks=pd.DataFrame()
    for cfg in config_files:
        log=config_to_log(cfg)
        if log is not None:
            ed=load_config(cfg)
            metrics=load_log(log,tags)
            for key,value in metrics.items():
                ed[key]=value
                
            task=edict_to_pandas(ed)
            tasks=tasks.append(task,ignore_index=True,sort=False)
        else:
            print('cannot find log file for',cfg)
            
    return tasks

rootpath=os.path.expanduser('~/tmp/logs/pytorch')
tags=['train/iou','val/iou']

/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


['/home/yzbx/git/torchseg', '', '/home/yzbx/git/torchseg/notebooks', '/home/yzbx/git/gnu/models/research', '/home/yzbx/git/gnu/models/research/slim', '/home/yzbx/bin/miniconda3/envs/new/lib/python36.zip', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/lib-dynload', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages/cityscapesscripts-1.0.0-py3.6-linux-x86_64.egg', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages/IPython/extensions', '/home/yzbx/.ipython']


In [11]:
rootpath=os.path.expanduser('~/tmp/logs/pytorch')
tags=['train/iou','val/iou']
filters=['caffe','pytorch','cityscapes','-1,1','0,1']

group_str='note'
group_tags=tags.copy()
group_tags.append(group_str)
    
tasks=summary(rootpath,tags)
filter_tasks=pd.DataFrame()
for filter_str in filters:
    filter_tasks=filter_tasks.append(tasks[tasks[group_str]==filter_str][group_tags],ignore_index=True,sort=False)

print(filter_tasks.to_string())

   train/iou   val/iou        note
0   0.459440  0.403973       caffe
1   0.460660  0.405963     pytorch
2   0.463188  0.416964  cityscapes
3   0.460112  0.400258        -1,1
4   0.460654  0.401508         0,1


In [45]:
rootpath=os.path.expanduser('~/tmp/logs/pytorch')
tags=['train/iou','val/iou']
configs=[
    {'filter_str':'hyperopt004',
    'group_str':'norm_ways'},
    {'filter_str':'hyperopt005',
    'group_str':'l2_reg'},
    {'filter_str':'hyperopt006',
    'group_str':'l2_reg'},
    {'filter_str':'hyperopt007',
    'group_str':'norm_ways'},
    {'filter_str':'hyperopt008',
    'group_str':'norm_ways'},
    {'filter_str':'hyperopt009',
    'group_str':'norm_ways'},
    {'filter_str':'hyperopt010',
    'group_str':'norm_ways'},
]

pd_strs=[]
for config in configs:
    tasks=summary(rootpath,tags,config['filter_str'])
    group_str=config['group_str']
    group_tags=tags.copy()
    group_tags.append(group_str)
    max_str=tasks[group_tags].groupby([group_str]).max().to_string()
    mean_str=tasks[group_tags].groupby([group_str]).mean().to_string()
    
    pd_strs.append((max_str,mean_str))

In [6]:
for d,(max_str,mean_str) in zip(configs,pd_strs):
    print('max',d['filter_str'],d['group_str'],'*'*30)
    print(max_str)
    print('mean',d['filter_str'],d['group_str'],'*'*30)
    print(mean_str)

max hyperopt004 norm_ways ******************************
            train/iou   val/iou
norm_ways                      
-1,1         0.737600  0.222334
0,1          0.733858  0.206601
caffe        0.737186  0.214802
cityscapes   0.743215  0.231846
pytorch      0.730023  0.208678
mean hyperopt004 norm_ways ******************************
            train/iou   val/iou
norm_ways                      
-1,1         0.731978  0.186179
0,1          0.728715  0.189355
caffe        0.711750  0.191525
cityscapes   0.727899  0.200267
pytorch      0.727013  0.187163
max hyperopt005 l2_reg ******************************
        train/iou   val/iou
l2_reg                     
0.0001   0.745121  0.221688
0.001    0.725888  0.221025
0.01     0.711914  0.220541
0.1      0.533035  0.197450
1e-05    0.738100  0.209281
mean hyperopt005 l2_reg ******************************
        train/iou   val/iou
l2_reg                     
0.0001   0.727874  0.191264
0.001    0.721769  0.185075
0.01     0.694604  0

In [53]:
notes=['wo_lr_mult','w_lr_mult','nw_cityscapes']
rootpath=os.path.expanduser('~/tmp/logs/pytorch')
tags=['train/iou','val/iou']

show_tags=tags.copy()
show_tags.append('note')
for note in notes:
    tasks=summary(rootpath,tags,note)
    print(tasks[show_tags].to_string())
    


   train/iou   val/iou        note
0    0.73829  0.526092  wo_lr_mult
   train/iou   val/iou       note
0   0.718299  0.532547  w_lr_mult
   train/iou   val/iou           note
0   0.788829  0.529173  nw_cityscapes


In [ ]:
notes=['cityscapes/cityscapes']
show_tags=tags.copy()
show_tags.append('note')
show_tags.append('learning_rate')
for note in notes:
    tasks=summary(rootpath,tags,note)
    print(tasks[show_tags].sort_values('val/iou').to_string())


In [ ]:
notes=['hyperopt011']
show_tags=tags.copy()
show_tags.append('note')
show_tags.append('use_lr_mult')
for note in notes:
    tasks=summary(rootpath,tags,note)
    print(tasks[show_tags].sort_values('val/iou').to_string())

In [74]:
notes=['hyperopt012','hyperopt013']
show_tags=[]
show_tags=tags.copy()
show_tags.append('note')
show_tags.append('l1_reg')
show_tags.append('l2_reg')
show_tags.append('learning_rate')
for note in notes:
    tasks=summary(rootpath,tags,note)
    tasks['learning_rate']=tasks['learning_rate']*float(10**4)
    print(tasks[show_tags].sort_values('val/iou').to_string())

    train/iou   val/iou         note       l1_reg       l2_reg  learning_rate
42   0.330699  0.246900  hyperopt012  1.20756e-05  5.19419e-05      39.978041
39   0.309704  0.254502  hyperopt012  3.12072e-05  3.44768e-06      49.488407
11   0.419695  0.267556  hyperopt012  2.97143e-06  0.000673939      19.785350
16   0.357555  0.273255  hyperopt012  2.65235e-07  7.84155e-06      32.902678
18   0.394275  0.297194  hyperopt012  2.12853e-05  1.41089e-05      27.622648
33   0.538827  0.309125  hyperopt012  0.000238267   0.00027026      15.335193
37   0.695417  0.348852  hyperopt012  0.000177286  8.31589e-06       9.022569
24   0.733858  0.349068  hyperopt012  0.000994241  1.06206e-07       7.908760
47   0.733766  0.351638  hyperopt012  0.000220961  2.23658e-06       6.103369
5    0.755240  0.357506  hyperopt012  2.90612e-07  0.000935574       3.792917
31   0.792348  0.362193  hyperopt012        0.001  1.22799e-05       1.000000
13   0.753151  0.362380  hyperopt012  1.11975e-07  0.000983984  

In [71]:
notes=['hyperopt']
show_tags=[]
show_tags=tags.copy()
show_tags.append('note')
show_tags.append('dataset_use_part')
for note in notes:
    tasks=summary(rootpath,tags,note)
    print(tasks[show_tags].groupby(['dataset_use_part','note']).max().to_string())

                              train/iou   val/iou
dataset_use_part note                            
0                hyperopt      0.467577  0.402875
1                hyperopt002   0.800826  0.228747
                 hyperopt004   0.743215  0.231846
                 hyperopt005   0.745121  0.221688
32               hyperopt008   0.743829  0.223043
                 hyperopt009   0.719986  0.215195
                 hyperopt010   0.684939  0.154941
                 hyperopt011   0.748290  0.215824
320              hyperopt006   0.797486  0.381637
                 hyperopt007   0.797381  0.392324
                 hyperopt012   0.805527  0.385403
                 hyperopt014   0.811780  0.386256
640              hyperopt013   0.825185  0.454330


In [69]:
from tabulate import tabulate
notes=['bn2_sgd200_nw','bn12','_wo_']
show_tags=[]
show_tags=tags.copy()
group_tags=['note']
show_tags+=group_tags
show_tags+=['n_epoch']
for note in notes:
    tasks=summary(rootpath,tags,note)
    #print(tasks[show_tags].groupby(group_tags).max().to_string())
    print(tabulate(tasks[show_tags].groupby(group_tags).max(),tablefmt='pipe',headers='keys'))
    print(tabulate(tasks[show_tags].sort_values('val/iou'),tablefmt='pipe',headers='keys'))

| note                 |   train/iou |   val/iou |   n_epoch |
|:---------------------|------------:|----------:|----------:|
| bn2_sgd200_nwcaffe   |    0.847756 |  0.55307  |       200 |
| bn2_sgd200_nwcity    |    0.842374 |  0.544636 |       200 |
| bn2_sgd200_nwpytorch |    0.822187 |  0.523619 |       200 |
|    |   train/iou |   val/iou | note                 |   n_epoch |
|---:|------------:|----------:|:---------------------|----------:|
|  0 |    0.822187 |  0.523619 | bn2_sgd200_nwpytorch |       200 |
|  1 |    0.842374 |  0.544636 | bn2_sgd200_nwcity    |       200 |
|  2 |    0.847756 |  0.55307  | bn2_sgd200_nwcaffe   |       200 |
| note            |   train/iou |   val/iou |   n_epoch |
|:----------------|------------:|----------:|----------:|
| bn12_nw_caffe   |    0.663704 |  0.516348 |        30 |
| bn12_nw_pytorch |    0.660641 |  0.524681 |        30 |
|    |   train/iou |   val/iou | note            |   n_epoch |
|---:|------------:|----------:|:----------------|

In [20]:
from tabulate import tabulate
notes=['014','015']
note_gtags=['learning_rate','norm_ways']
describe=[]
for note,ntag in zip(notes,note_gtags):
    show_tags=[]
    show_tags=tags.copy()
    group_tags=['note',ntag]
    show_tags+=group_tags
    show_tags+=['n_epoch']
    tasks=summary(rootpath,tags,note)
    #print(tasks[show_tags].groupby(group_tags).max().to_string())
    print(tabulate(tasks[show_tags].groupby(group_tags).mean().sort_values('val/iou'),tablefmt='pipe',headers='keys'))
    print(tabulate(tasks[['val/iou']+group_tags].groupby(group_tags).agg([np.mean,np.std,np.max]),tablefmt='pipe',headers='keys'))
    print(tabulate(tasks[show_tags].sort_values('val/iou'),tablefmt='pipe',headers='keys'))

|                          |   train/iou |   val/iou |   n_epoch |
|:-------------------------|------------:|----------:|----------:|
| ('hyperopt014', '1e-05') |    0.731833 |  0.323429 |       100 |
| ('hyperopt014', '2e-05') |    0.747784 |  0.334623 |       100 |
| ('hyperopt014', 0.001)   |    0.684718 |  0.342896 |       100 |
| ('hyperopt014', '5e-05') |    0.758231 |  0.359602 |       100 |
| ('hyperopt014', 0.0005)  |    0.756845 |  0.363612 |       100 |
| ('hyperopt014', 0.0001)  |    0.782894 |  0.370021 |       100 |
| ('hyperopt014', 0.0002)  |    0.799771 |  0.371424 |       100 |
|                          |   ('val/iou', 'mean') |   ('val/iou', 'std') |   ('val/iou', 'amax') |
|:-------------------------|----------------------:|---------------------:|----------------------:|
| ('hyperopt014', 0.0001)  |              0.370021 |           0.00833878 |              0.382354 |
| ('hyperopt014', 0.0002)  |              0.371424 |           0.0103945  |              0.386256

In [66]:
g_tasks=tasks[['val/iou']+group_tags].groupby(group_tags).agg([np.mean,np.std,np.max])
print(g_tasks)
print(g_tasks.columns)
print(g_tasks.keys)
print(g_tasks['val/iou'].columns)
print(g_tasks['val/iou']['mean'])
print(g_tasks['val/iou','std'])
print(g_tasks.loc[('wo_bn_015','0,1'),('val/iou','mean')])

g_tasks.reset_index(inplace=True)
print(g_tasks)

                       val/iou                    
                          mean       std      amax
note      norm_ways                               
wo_bn_015 -1,1        0.365011  0.005917  0.371074
          0,1         0.372147  0.003111  0.375468
          caffe       0.372286  0.007410  0.381298
          cityscapes  0.370536  0.005781  0.375322
          pytorch     0.370970  0.005323  0.374734
MultiIndex(levels=[['val/iou'], ['mean', 'std', 'amax']],
           labels=[[0, 0, 0], [0, 1, 2]])
<bound method NDFrame.keys of                        val/iou                    
                          mean       std      amax
note      norm_ways                               
wo_bn_015 -1,1        0.365011  0.005917  0.371074
          0,1         0.372147  0.003111  0.375468
          caffe       0.372286  0.007410  0.381298
          cityscapes  0.370536  0.005781  0.375322
          pytorch     0.370970  0.005323  0.374734>
Index(['mean', 'std', 'amax'], dtype='object')
note  

In [67]:
g_tasks=g_tasks.transpose()
g_tasks.reset_index(inplace=True)
print(g_tasks)

     level_0 level_1           0           1           2           3  \
0       note           wo_bn_015   wo_bn_015   wo_bn_015   wo_bn_015   
1  norm_ways                -1,1         0,1       caffe  cityscapes   
2    val/iou    mean    0.365011    0.372147    0.372286    0.370536   
3    val/iou     std  0.00591723  0.00311127  0.00741026  0.00578118   
4    val/iou    amax    0.371074    0.375468    0.381298    0.375322   

            4  
0   wo_bn_015  
1     pytorch  
2     0.37097  
3  0.00532321  
4    0.374734  


In [68]:
g_tasks.reset_index?

Signature: g_tasks.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')
Docstring:
For DataFrame with multi-level index, return new DataFrame with
labeling information in the columns under the index names, defaulting
to 'level_0', 'level_1', etc. if any are None. For a standard index,
the index name will be used (if set), otherwise a default 'index' or
'level_0' (if 'index' is already taken) will be used.

Parameters
----------
level : int, str, tuple, or list, default None
    Only remove the given levels from the index. Removes all levels by
    default
drop : boolean, default False
    Do not try to insert index into dataframe columns. This resets
    the index to the default integer index.
inplace : boolean, default False
    Modify the DataFrame in place (do not create a new object)
col_level : int or str, default 0
    If the columns have multiple levels, determines which level the
    labels are inserted into. By default it is inserted into the first
    